# Load a model and generate poems

This notebook has two objectives:
    - load a trained poem generator
    - load a trained rhyme generator
    - Generate some poetry
    

## Importing packages and loading models
We start by importing a couple of packages and load models previously trained

In [5]:
import tensorflow as tf  # version 1.9 or above
tf.enable_eager_execution()  # Execution of code as it runs in the notebook. Normally, TensorFlow looks up the whole code before execution for efficiency.
from tensorflow.keras.layers import Embedding, GRU, Dense
import numpy as np
import re
from tensorflow.train import AdamOptimizer
from tensorflow.losses import sparse_softmax_cross_entropy
import time

## Load models and hyperparameters

Now we can load hyperparameters for both the poem generator and the rhyme generator

Let's start with the poem generator

In [6]:
# Load hyperparameters and layers' weights previously saved
parameters_poems = np.load('model_poems.npy')[()]
# get the pretrained weights
embedding_weights_poems = parameters_poems['embedding_weights']
gru_weights_poems = parameters_poems['gru_weights']
fc_weights_poems = parameters_poems['fc_weights']

# dictionaries character to integer / integer to character
char2idx_poems = parameters_poems['char2idx']
idx2char_poems = parameters_poems['idx2char']

# Hyperparameters
max_length_poems = parameters_poems['max_length']  # Maximum length sentence we want per input in the network
embedding_dim_poems = parameters_poems['embedding_dim']  # number of 'meaningful' features to learn. Ex: ['queen', 'king', 'man', 'woman'] has a least 2 embedding dimension: royalty and gender.
units_poems = parameters_poems['units']  # In keras: number of output of a sequence. In short it rem
BATCH_SIZE_poems = parameters_poems['BATCH_SIZE']
BUFFER_SIZE_poems = parameters_poems['BUFFER_SIZE']

vocab_size_poems = len(dict(idx2char_poems))

Now let's load the hyperparameters and weights for the rhyme generator

In [7]:
# Load hyperparameters and layers' weights previously saved
parameters_rhymes = np.load('model_rhymes.npy')[()]
# get the pretrained weights
embedding_weights_rhymes = parameters_rhymes['embedding_weights']
gru_weights_rhymes = parameters_rhymes['gru_weights']
fc_weights_rhymes = parameters_rhymes['fc_weights']

# dictionaries character to integer / integer to character
word2idx_rhymes = parameters_rhymes['word2idx']
idx2word_rhymes = parameters_rhymes['idx2word']

# Hyperparameters
max_length_rhymes = parameters_rhymes['max_length']  # Maximum length sentence we want per input in the network
embedding_dim_rhymes = parameters_rhymes['embedding_dim']  # number of 'meaningful' features to learn. Ex: ['queen', 'king', 'man', 'woman'] has a least 2 embedding dimension: royalty and gender.
units_rhymes = parameters_rhymes['units']  # In keras: number of output of a sequence. In short it rem
BATCH_SIZE_rhymes = parameters_rhymes['BATCH_SIZE']
BUFFER_SIZE_rhymes = parameters_rhymes['BUFFER_SIZE']

vocab_size_rhymes = len(dict(idx2word_rhymes))

## Models creation

We now reproduce models with the same structure as the ones previously trained. 

*Note: There is no need for declaring two classes (one for proems, the other for rhymes). Indeed, both poems and rhymes models are based on the same architecture.*

In [8]:

class Model(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, units, batch_size):
        super(Model, self).__init__()
        self.units = units
        self.batch_sz = batch_size
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.gru = GRU(self.units, return_sequences=True, return_state=True, recurrent_activation='sigmoid', recurrent_initializer='glorot_uniform')
        self.fc = Dense(vocab_size)

    def call(self, x, hidden):
        x = self.embedding(x)
        output, states = self.gru(x, initial_state=hidden)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, states

    
model_poems = Model(vocab_size_poems, embedding_dim_poems, units_poems, BATCH_SIZE_poems)
model_rhymes = Model(vocab_size_rhymes, embedding_dim_rhymes, units_rhymes, BATCH_SIZE_rhymes)

## Setting pretrained weights

Let's plug our pretrained weights in our poem generator.
We need to predict a blank character (I ignore the reason) before we may set the weights we've trained 

In [9]:
num_generate = 1
start_string = 'child'[::-1]
input_eval = [char2idx_poems[s] for s in start_string]
input_eval = tf.expand_dims(input_eval, 0)
hidden = [tf.zeros((1, units_poems))]
predictions, hidden = model_poems(input_eval, hidden)

model_poems.embedding.set_weights(np.asarray(embedding_weights_poems))
model_poems.gru.set_weights(gru_weights_poems)
model_poems.fc.set_weights(fc_weights_poems)

We repeat the process with the rhyme generator

In [10]:
num_generate = 1  # number of characters to generate
start_string = ['fell']  # beginning of the generated text. TODO: try start_string = ' '
input_eval = [word2idx_rhymes[s] for s in start_string]  # converts start_string to numbers the model understands
input_eval = tf.expand_dims(input_eval, 0)
hidden = [tf.zeros((1, units_rhymes))]
predictions, hidden = model_rhymes(input_eval, hidden)

model_rhymes.embedding.set_weights(np.asarray(embedding_weights_rhymes))
model_rhymes.gru.set_weights(gru_weights_rhymes)
model_rhymes.fc.set_weights(fc_weights_rhymes)

## Text Generation

Finally, we can generate some poetry text from our model.

First the rhymes:

In [11]:
temperature = 0.4

num_generate = 4  # number of characters to generate
start_string = ['fell', 'vain', 'well', 'tree', 'fell', 'leave', 'me', 'above', 'melody']  # beginning of the generated text. TODO: try start_string = ' '
input_eval = [word2idx_rhymes[s] for s in start_string]  # converts start_string to numbers the model understands
input_eval = tf.expand_dims(input_eval, 0) 

text_generated = []


hidden = [tf.zeros((1, units_rhymes))]
for i in range(num_generate):
    predictions, hidden = model_rhymes(input_eval, hidden)  # predictions holds the probabily for each character to be most adequate continuation
   
    predictions = predictions / temperature  # alters characters' probabilities to be picked (but keeps the order)
    predicted_id = tf.multinomial(tf.exp(predictions), num_samples=1)[0][0].numpy()  # picks the next character for the generated text
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated += [idx2word_rhymes[predicted_id]]

rhymes = text_generated

Then, let's generate some poetry

In [12]:
# Evaluation step(generating text using the model learned)


for temperature in [0.5, 0.7, 0.9, 1.1, 1.5]:
    print('Temperature = {} \n'.format(temperature))
    text_generated = ''
    for rhyme in rhymes:
    
        num_generate = 150  # number of characters to generate
        start_string = text_generated + rhyme[::-1]  # beginning of the generated text. TODO: try start_string = ' '
        input_eval = [char2idx_poems[s] for s in start_string]  # converts start_string to numbers the model understands
        input_eval = tf.expand_dims(input_eval, 0)  # 
        hidden = [tf.zeros((1, units_poems))]
        
        b = True
        c = 1
        added_text = ''
        while b == True:
            predictions, hidden = model_poems(input_eval, hidden)  # predictions holds the probabily for each character to be most adequate continuation
           
            predictions = predictions / temperature  # alters characters' probabilities to be picked (but keeps the order)
            predicted_id = tf.multinomial(tf.exp(predictions), num_samples=1)[0][0].numpy()  # picks the next character for the generated text
            input_eval = tf.expand_dims([predicted_id], 0)
            added_text += idx2char_poems[predicted_id]
            c += 1
            if idx2char_poems[predicted_id] == '\n' or c > num_generate:
                text_generated += rhyme[::-1] + added_text
                b = False
    print('Poem : \n')
    print (text_generated[::-1])
    print('\n')

Temperature = 0.5 

Poem : 


there  when seas alive  in such numbers had watched on the flood
he knew                                                                                                           was it shattered  then   never  so swiftly grew
too soon that comes to the land and blood


Temperature = 0.7 

Poem : 


there  when seas alive  in such numbers had watched on the flood
he knew                                                                                                           was it shattered  then   never  so swiftly grew
too soon that comes to the land and blood


Temperature = 0.9 

Poem : 


as for god s eyes are heaped and flood
i live in my mother  for he never knew
and then he smiled on his nose up grew
the thunder steeds  and strength and blood


Temperature = 1.1 

Poem : 


this is the same as on the flood
so she looked down by her brother knew
and said   no  never  so swiftly grew
too soon that comes to the land and blood


Temperature = 1.5 

Po